In [264]:
import xgboost as xgb
import pandas as pd
import numpy as np
import json
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.feature_selection import SelectPercentile, f_classif, chi2
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectKBest, f_regression

In [406]:


# with open('train.json') as f:
    # data = json.load(f)

df = pd.read_json('train.json')
# Use pd.json_normalize to convert the JSON to a DataFrame
# df = pd.json_normalize(data, 
#                      meta=['marca', 'model'])
# alladdons = []
# for index, row in df.iterrows():    

#     for x in row['addons']:
#         if x not in alladdons:
#             alladdons.append(x)

# for addon in alladdons:
#     df[addon] = df['addons'].apply(lambda addons_list: 1 if addon in addons_list else 0)
    
# print(df['addons'])


# # Rename the columns for clarity
# df.columns = ['Marca', 'model', 'an', 'km', 'putere','cutie_de_viteze','combustibil','capacitate_cilindrica','transmisie','caroserie','culoare','optiuni_culoare']

# Display the DataFrame
# print(df)

In [407]:
# Presupunând că df este DataFrame-ul tău cu datele
# X reprezintă caracteristicile, iar y reprezintă variabila țintă (în acest caz, 'pret')
df["nr_opt"] = df['addons'].apply(lambda addons_list: len(addons_list))
df['km_standardized'] = (df['km'] - df['km'].mean()) / df['km'].std()
# df['km_standardized'] =(df['km'] - df['km'].min()) / (df['km'].max() - df['km'].min())
# df['putere_standardized'] = (df['putere'] - df['putere'].mean()) / df['putere'].std()
df['capacitate_cilindrica_standardized'] = (df['capacitate_cilindrica'] - df['capacitate_cilindrica'].mean()) / df['capacitate_cilindrica'].std()
df['age'] = 2024 - df['an']
# df = df.drop('an', axis=1)
df = df.drop('km', axis=1)
# df = df.drop('putere', axis=1)
df = df.drop('model', axis=1)
df = df.drop('marca', axis=1)
df = df.drop('capacitate_cilindrica', axis=1)
# df = df[df['addons'].apply(len) > 1]
df = df.drop('addons', axis=1)
# df = df.drop('id', axis=1)
X = df.drop('pret', axis=1)  # Excludem 'pret' din caracteristici
y = df['pret']  # 'pret' este variabila țintă

X_encoded = pd.get_dummies(X, columns=['cutie_de_viteze', 'combustibil', 'transmisie', 'caroserie', 'culoare', 'optiuni_culoare'])

# Împărțim setul de date în seturi de antrenare și testare (de exemplu, 80% antrenare, 20% testare)
X_train_encoded, X_test_encoded, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Afișăm dimensiunile seturilor rezultate
# print("Dimensiunile setului de antrenare (X_train, y_train):", X_train.shape, y_train.shape)
# print("Dimensiunile setului de testare (X_test, y_test):", X_test.shape, y_test.shape)


In [408]:


model = XGBRegressor(n_estimators=250, learning_rate=0.2, max_depth=6, random_state=42)
model.set_params(
    objective='reg:squarederror',
    colsample_bytree=0.8,
)
model.fit(X_train_encoded, y_train)
# Best Hyperparameters: {'learning_rate': 0.4, 'max_depth': 5, 'n_estimators': 200}

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.2, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=250, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=42, ...)

In [409]:
y_pred = model.predict(X_test_encoded)

In [410]:
r_squared = r2_score(y_test, y_pred)
print(f"R-squared Score: {r_squared}")


R-squared Score: 0.9385286598430622


In [411]:
dftest = pd.read_json('test.json')

dftest["nr_opt"] = dftest['addons'].apply(lambda addons_list: len(addons_list))
dftest['km_standardized'] = (dftest['km'] - dftest['km'].mean()) / dftest['km'].std()
# df['km_standardized'] =(df['km'] - df['km'].min()) / (df['km'].max() - df['km'].min())
# df['putere_standardized'] = (df['putere'] - df['putere'].mean()) / df['putere'].std()
dftest['capacitate_cilindrica_standardized'] = (dftest['capacitate_cilindrica'] - dftest['capacitate_cilindrica'].mean()) / dftest['capacitate_cilindrica'].std()
dftest['age'] = 2024 - dftest['an']
# df = df.drop('an', axis=1)
dftest = dftest.drop('km', axis=1)
# df = df.drop('putere', axis=1)
dftest = dftest.drop('model', axis=1)
dftest = dftest.drop('marca', axis=1)
dftest = dftest.drop('capacitate_cilindrica', axis=1)
# dftest = dftest[dftest['addons'].apply(len) > 1]
dftest = dftest.drop('addons', axis=1)
# dftest = dftest.drop('id', axis=1)
Xt = dftest.drop('pret', axis=1)  # Excludem 'pret' din caracteristici
yt = dftest['pret']  # 'pret' este variabila țintă

X_encoded_t = pd.get_dummies(Xt, columns=['cutie_de_viteze', 'combustibil', 'transmisie', 'caroserie', 'culoare', 'optiuni_culoare'])

In [412]:
pred = model.predict(X_encoded_t)


In [413]:
newdftest = pd.read_json('test.json')
newdftest['pret'] = pred

# write test.json from newdftest
newdftest.to_json('test.json', orient='records')

In [258]:
print(model.feature_importances_)

[1.24395648e-02 8.58985782e-02 2.86931619e-02 5.91022521e-02
 2.92040902e-04 0.00000000e+00 8.56979295e-06 2.71289009e-05
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 6.00347004e-04 0.00000000e+00 0.00000000e+00
 4.61715972e-03 1.32889894e-03 1.85462108e-04 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 8.17318214e-04
 2.09623715e-03 0.00000000e+00 2.12992751e-03 1.73989274e-02
 0.00000000e+00 9.26597917e-04 3.20601207e-03 0.00000000e+00
 8.44726339e-04 2.36558681e-03 1.23212428e-03 1.08536542e-03
 8.21227022e-03 2.94127618e-04 0.00000000e+00 3.80174984e-04
 0.00000000e+00 0.00000000e+00 0.00000000e+00 7.16888637e-04
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 3.44641216e-04 3.02886911e-04 0.00000000e+00
 3.58449179e-04 0.00000000e+00 0.00000000e+00 0.00000000e+00
 4.97208443e-04 1.33764802e-03 0.00000000e+00 1.40672573e-03
 2.94005556e-04 8.27890879e-04 2.88692332e-04 1.79025473e-03
 3.92807228e-03 2.966796

In [252]:
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 20784037.082104824


In [240]:
param_grid = {
    'n_estimators': [100, 200,250, 300, 400, 500],
    'learning_rate': [0.001,0.1, 0.2, 0.3, 0.4],
    'max_depth': [4, 5, 6, 7, 8]
}

grid_search = GridSearchCV(estimator=XGBRegressor(random_state=42),
                           param_grid=param_grid,
                           scoring='r2',
                           cv=2,
                           verbose=3)

grid_search.fit(X_train_encoded, y_train)

best_params = grid_search.best_params_
print(f"Best Hyperparameters: {best_params}")

# Use the best model for predictions
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test_encoded)

Fitting 2 folds for each of 150 candidates, totalling 300 fits
[CV 1/2] END learning_rate=0.001, max_depth=4, n_estimators=100;, score=-0.925 total time=   1.0s
[CV 2/2] END learning_rate=0.001, max_depth=4, n_estimators=100;, score=-0.925 total time=   0.9s
[CV 1/2] END learning_rate=0.001, max_depth=4, n_estimators=200;, score=-0.608 total time=   1.8s
[CV 2/2] END learning_rate=0.001, max_depth=4, n_estimators=200;, score=-0.606 total time=   1.8s
[CV 1/2] END learning_rate=0.001, max_depth=4, n_estimators=250;, score=-0.472 total time=   2.2s
[CV 2/2] END learning_rate=0.001, max_depth=4, n_estimators=250;, score=-0.468 total time=   4.4s
[CV 1/2] END learning_rate=0.001, max_depth=4, n_estimators=300;, score=-0.348 total time=   3.1s
[CV 2/2] END learning_rate=0.001, max_depth=4, n_estimators=300;, score=-0.343 total time=   3.0s
[CV 1/2] END learning_rate=0.001, max_depth=4, n_estimators=400;, score=-0.133 total time=   3.8s
[CV 2/2] END learning_rate=0.001, max_depth=4, n_estima

In [ ]:
r_squared = r2_score(y_test, y_pred_best)
print(f"R-squared Score: {r_squared}")
# 0.9581180349227657 doar cu addons-urile sterse
# 0.9523361370125374 cu addonsurile la coloane
